In [1]:

import numpy as np

from DataAnalysisClass import *
from clustering_work import *

single = ['aug-cc-pVDZ', 'aug-cc-pVTZ', 'aug-cc-pVQZ', 'aug-cc-pV5Z', 'aug-cc-pV6Z']
single_polarized = ['aug-cc-pCVDZ', 'aug-cc-pCVTZ', 'aug-cc-pCVQZ']
double = ['d-aug-cc-pVDZ', 'd-aug-cc-pVTZ', 'd-aug-cc-pVQZ', 'd-aug-cc-pV5Z', 'd-aug-cc-pV6Z']
double_polarized = ['d-aug-cc-pCVDZ', 'd-aug-cc-pCVTZ', 'd-aug-cc-pCVQZ']
all_basis_sets = single + single_polarized + double + double_polarized


In [2]:

database_path = Path('/home/ahurta92/data/august')
paper_path = Path('response_paper_figures')



In [3]:
import glob

# glob for .mol files in august molecules directory
mols = glob.glob('/home/ahurta92/data/august/molecules/*.mol')
mols = [mol.split('/')[-1].split('.')[0] for mol in mols]
mols.remove('LiH_s')
mols


In [4]:
august_database = PolarizabilityData(mols, 'hf', 'dipole', all_basis_sets, database_path, overwrite=False)

In [5]:
august_database.save_dfs()

In [6]:
polar_data = august_database.iso_data.copy()

In [7]:
eigen_data = MRAComparedBasisDF(august_database.alpha_eigen, ['molecule', 'omega', 'ij'], ['alpha'], True)
eigen_data.set_index(['molecule', 'omega', 'ij', 'basis'], inplace=True)
eigen_data = eigen_data['alphaE']
eigen_data = eigen_data.reset_index()
eigen_data


In [8]:
mol_datas = []
for mol in eigen_data.molecule.unique():
    mol_data = eigen_data.query('molecule==@mol and omega==0 and ij.isin(["xx", "yy", "zz"])')
    mol_data.set_index(['basis', 'ij'], inplace=True)
    mol_data = pd.Series(mol_data['alphaE'], name=mol)
    mol_datas.append(mol_data)

mol_data = pd.concat(mol_datas, axis=1)
eigen_data = mol_data.T.dropna()
eigen_data

In [9]:

basis_data = MRAComparedBasisDF(polar_data, ['molecule', 'omega'], ['alpha', 'gamma'], True)


def get_basis_data(basis_data, omega=8):
    df = basis_data.query('omega==@omega')
    alphab = {}
    gammab = {}
    for b in basis_data.basis.unique():
        bdata = df.query('basis==@b')
        alphab[b] = bdata.set_index('molecule').alphaE
        gammab[b] = bdata.set_index('molecule').gammaE
    alpha_df = pd.DataFrame(alphab)
    gamma_df = pd.DataFrame(gammab)
    return alpha_df, gamma_df


data = get_basis_data(basis_data, 0)[0]
data.drop(['aug-cc-pV5Z', 'aug-cc-pV6Z', 'd-aug-cc-pV5Z', 'd-aug-cc-pV6Z'], axis=1, inplace=True)
data.dropna(inplace=True)


In [10]:
data

In [11]:

def cluster_basis_data(data, n_clusters=8):
    scaler = StandardScaler()
    X = data.to_numpy()
    data_scaled = scaler.fit_transform(X)
    # Create an AgglomerativeClustering instance with n_clusters
    agglo = AgglomerativeClustering(n_clusters=n_clusters)
    # Fit the model to your data and get the cluster assignments in one step
    labels = agglo.fit_predict(data_scaled)
    data['cluster'] = labels

    return data


def inv_symlog(y, linthresh):
    """Inverse of symmetric log transformation."""
    return np.sign(y) * linthresh * (np.exp(np.abs(y)) - 1)


def symlog(x, linthresh):
    """Symmetric log transformation."""

    copy_x = x.copy()
    copy_x[np.abs(x) < linthresh] = 0
    #return np.log(np.abs(copy_x/linthresh))
    #return np.sign(copy_x) * np.log1p(np.abs(copy_x / linthresh))
    return np.sign(copy_x) * np.log1p(np.abs(copy_x / linthresh))

In [12]:

from sklearn import preprocessing

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.mixture import GaussianMixture
# plot data_matrix for each column
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [14]:
data = get_basis_data(basis_data, 0)[0]
#data= mol_data.T
data.drop(['aug-cc-pV5Z', 'aug-cc-pV6Z', 'd-aug-cc-pV5Z', 'd-aug-cc-pV6Z'], axis=1, inplace=True)
# drop DZ basis sets
#data.drop(['aug-cc-pVDZ', 'd-aug-cc-pVDZ','aug-cc-pCVDZ','d-aug-cc-pCVDZ'], axis=1, inplace=True)
#data.drop(['aug-cc-pVTZ', 'd-aug-cc-pVTZ','aug-cc-pCVTZ','d-aug-cc-pCVTZ'], axis=1, inplace=True)
data.dropna(inplace=True)
data



In [15]:
threshold = 5e-2  # threshold for symlog transformation           

quantile = preprocessing.QuantileTransformer(n_quantiles=10)
robust = preprocessing.RobustScaler()
normalizer = preprocessing.Normalizer()
standard = preprocessing.StandardScaler()

data_matrix = data.to_numpy()
# normalize all values to be between +1 and -1 
#data_matrix/=np.max(np.abs(data_matrix))


data_matrix = symlog(data_matrix, threshold)
#data_matrix = quantile.fit_transform(data_matrix)
#data_matrix=standard.fit_transform(data_matrix)
data_matrix = normalizer.fit_transform(data_matrix)

fig, axes = plt.subplots(4, 3, figsize=(18, 6), sharex=True, sharey=True)
for i, ax in enumerate(axes.flat):
    sns.histplot(data_matrix[:, i], ax=ax, kde=False, bins=10)
    ax.set_title(data.columns[i])


def gmm_bic_score(estimator, X):
    return -estimator.bic(X)

In [130]:



param_grid = {
    'n_components': range(1, 16),
    'init_params': ['kmeans', 'random_from_data'],  #
    'covariance_type': ['spherical']
}

grid_search = GridSearchCV(GaussianMixture(tol=1e-9, n_init=100, max_iter=2000),
                           param_grid=param_grid, scoring=gmm_bic_score, verbose=3, )
model = grid_search

X = data_matrix
#grid_search.fit(X)
model.fit(X)

df = pd.DataFrame(grid_search.cv_results_)[
    ["param_n_components", "param_covariance_type", "mean_test_score"]
]
df["mean_test_score"] = -df["mean_test_score"]
df = df.rename(
    columns={
        "param_n_components": "Number of components",
        "param_covariance_type": "Type of covariance",
        "mean_test_score": "BIC score",
    }
)

print(grid_search.best_params_)
df.sort_values(by="BIC score").head()

# plot the BIC score for each model
fig, ax = plt.subplots(figsize=(10, 6))
sns.lineplot(data=df, x="Number of components", y="BIC score", hue="Type of covariance", ax=ax)



In [52]:
threshold = 5e-2  # threshold for symlog transformation           

data = get_basis_data(basis_data, 0)[0]
#data= mol_data.T
data.drop(['aug-cc-pV5Z', 'aug-cc-pV6Z', 'd-aug-cc-pV5Z', 'd-aug-cc-pV6Z'], axis=1, inplace=True)
data.drop(['aug-cc-pVDZ', 'd-aug-cc-pVDZ', 'aug-cc-pCVDZ', 'd-aug-cc-pCVDZ'], axis=1, inplace=True)
data.drop(['aug-cc-pVTZ', 'd-aug-cc-pVTZ', 'aug-cc-pCVTZ', 'd-aug-cc-pCVTZ'], axis=1, inplace=True)
data.dropna(inplace=True)
normalizer = preprocessing.Normalizer()
standard = preprocessing.StandardScaler()

data_matrix = symlog(data_matrix, threshold)
data_matrix = normalizer.fit_transform(data_matrix)

num_clusters = 4

model = GaussianMixture(n_components=num_clusters, covariance_type='spherical', tol=1e-8, n_init=100, max_iter=2000,
                        init_params='kmeans', )
model.fit(data_matrix)

labels = model.predict(data_matrix)

color_iter = sns.color_palette("tab10", 2)[::-1]
#labels = gmm.predict(data_matrix)
score = silhouette_score(data_matrix, labels, metric='euclidean')
data['cluster'] = labels
print("silhouette score", score)

average_vectors = []
std_vectors = []
for i in range(num_clusters):
    cluster_points = data_matrix[labels == i]
    average_vector = np.mean(cluster_points, axis=0)
    std_vector = np.std(cluster_points, axis=0)
    average_vectors.append(average_vector)
    std_vectors.append(std_vector)

avg_df = pd.DataFrame(average_vectors, )
avg_df['mean'] = avg_df.mean(axis=1)
avg_df = avg_df.sort_values('mean', ascending=False)
avg_df.drop('mean', axis=1, inplace=True)

std_df = pd.DataFrame(std_vectors, )

sorted_index = avg_df.index
std_df = std_df.reindex(sorted_index)
std_df.sort_index(inplace=True)

cluster_map = {sorted_index[i]: i for i in range(len(sorted_index))}
avg_df = avg_df.reset_index(drop=True)
# avg_df = avg_df.apply(lambda x: inv_symlog(x, threshold))
# avg_df = pd.DataFrame(scaler.inverse_transform(avg_df), columns=avg_df.columns,
#                      index=avg_df.index)
print(cluster_map)
data['cluster'] = data['cluster'].map(cluster_map)

iso_diff = basis_data.copy()
iso_diff['cluster'] = iso_diff['molecule'].map(data['cluster'])
iso_diff['cluster'] = iso_diff['cluster'] + 1
iso_diff['cluster'] = iso_diff['cluster'].astype('category')

iso_diff.dropna(inplace=True)



In [53]:
plot_data = iso_diff.query('omega==0')

basis_labels = ['aug-pVDZ', 'aug-pVTZ', 'aug-pVQZ', 'aug-pCVDZ', 'aug-pCVTZ', 'aug-pCVQZ', 'd-aug-pVDZ', 'd-aug-pVTZ',
                'd-aug-pVQZ', 'd-aug-pCVDZ',
                'd-aug-pCVTZ', 'd-aug-pCVQZ']
basis_order = ['aug-cc-pVDZ', 'aug-cc-pCVDZ', 'aug-cc-pVTZ', 'aug-cc-pCVTZ', 'aug-cc-pVQZ', 'aug-cc-pCVQZ',
               'd-aug-cc-pVDZ', 'd-aug-cc-pCVDZ', 'd-aug-cc-pVTZ',
               'd-aug-cc-pCVTZ', 'd-aug-cc-pVQZ', 'd-aug-cc-pCVQZ']
basis_order = ['aug-cc-pVDZ', 'aug-cc-pVTZ', 'aug-cc-pVQZ', 'aug-cc-pCVDZ', 'aug-cc-pCVTZ', 'aug-cc-pCVQZ',
               'd-aug-cc-pVDZ', 'd-aug-cc-pVTZ', 'd-aug-cc-pVQZ',
               'd-aug-cc-pCVDZ', 'd-aug-cc-pCVTZ', 'd-aug-cc-pCVQZ']
basis_labels = ['s-D', 's-T', 's-Q', 's-CD', 's-CT', 's-CQ', 'd-D', 'd-T', 'd-Q', 'd-CD', 'd-CT', 'd-CQ']
#basis_labels = ['s-D','s-CD', 's-T', 's-CT', 's-Q', 's-CQ', 'd-D', 'd-CD', 'd-T', 'd-CT', 'd-Q', 'd-CQ']

plot_data = plot_data.query(
    'basis!="aug-cc-pV5Z" and basis!="aug-cc-pV6Z" and basis!="d-aug-cc-pV5Z" and basis!="d-aug-cc-pV6Z"')
plot_data['basis'] = plot_data['basis'].astype('category')
plot_data['basis'] = plot_data['basis'].cat.reorder_categories(basis_order)
plot_data

In [58]:

from matplotlib.ticker import ScalarFormatter

sharey=True
if sharey:
    ci=0
else:
    ci = 95

g = sns.FacetGrid(plot_data, col='cluster', col_wrap=2, sharey=sharey, sharex=True, height=3, aspect=1.5)

g.map_dataframe(sns.lineplot,x='basis',y='alphaE', hue='Type', size=10, alpha=.75,palette=['k'],errorbar=('ci',ci), 
                legend=False)
g.map_dataframe(sns.stripplot,x='basis',y='alphaE', hue='mol_system', size=5, alpha=.75, dodge=False, palette='colorblind',
      legend=True)
g.set_ylabels(r'$\alpha$ Percent Error')

for i,ax in enumerate(g.axes.flat):
    j=i+1
    cluster_size = plot_data.query('cluster==@j').molecule.unique().size
    ax.set_title(f'Cluster {j} ({cluster_size}/ {plot_data.molecule.unique().size})')
    
    ax.axhline(y=threshold, linestyle='dashdot', color='red', linewidth=.50)
    ax.axhline(y=-threshold, linestyle='dashdot', color='red', linewidth=.50)
    ax.axhline(y=.00, linestyle='--', color='red', linewidth=.35)
    if sharey:
        ax.set_yscale('symlog', linthresh=.1)
        ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.set_xticks(range(0, 12))
    ax.set_xticklabels(basis_labels, rotation=45)
    ax.axvline(x=2.5, linestyle='dotted', color='black', linewidth=.15)
    ax.axvline(x=5.5, linestyle='dotted', color='black', linewidth=.15)
    ax.axvline(x=8.5, linestyle='dotted', color='black', linewidth=.15)
    ax.set_xlabel('')

handles, labels = ax.get_legend_handles_labels()
# add a legened
g.fig.legend(handles, labels, loc='center', bbox_to_anchor=(0.5, 0.00), ncol=3, fancybox=True, fontsize=10)

g.fig.tight_layout()
g.fig.savefig(paper_path.joinpath('alpha_convergence.png'), dpi=600, bbox_inches='tight')


In [59]:


cdata = iso_diff.dropna().query('omega==0 and basis=="aug-cc-pVDZ"')
cluster_table = cdata[['molecule', 'cluster', 'mol_system']].drop_duplicates().sort_values('mol_system')
mol_table = pd.DataFrame()
#mol_table.columns = [f'Cluster {i}' for i in range(1, len(cluster_table['cluster'].unique()))]

cluster_data = []
for cluster_id, group in cluster_table.groupby('cluster'):
    print(cluster_id)
    cs = pd.Series(group['molecule'])
    cs.name = f'Cluster {int(cluster_id)}'
    cs.reset_index(drop=True, inplace=True)
    cluster_data.append(cs)

mol_table = pd.concat(cluster_data, axis=1)
# sort the values of each column alphabetically
mol_table = mol_table.fillna('-')
mol_table = mol_table.applymap(lambda x: '\\ce{' + str(x) + '}', na_action='ignore')
# reverse the order to the table rowwise
mol_table = mol_table.iloc[::-1]
# map colorblind colormap to the columns
mol_only = cluster_table['molecule']
# make mol only into a table 9 x 10 

# surround the values with \ce{} to make them chemical formulas
ms = mol_only.apply(lambda x: r'\ce{' + str(x) + '}')
# make a dictionary of the colors
mol_dict = dict(zip(ms, cluster_table.mol_system))

pal = sns.color_palette('muted', n_colors=3)
colormap = dict(zip(cluster_table.mol_system.unique(), pal.as_hex()))
mol_color = {mol: colormap[mol_dict[mol]] for mol in ms}
# add white to \ce{-} to make it white
mol_color = {mol: colormap[mol_dict[mol]] for mol in ms}

mol_color['\\ce{-}'] = 'white'
# apply the colors to the dataframe based on mol_color dictionary

#mol_df = mol_df.applymap(lambda x: '\\ce{' + str(x) + '}')
mol_df = mol_table.style.apply(lambda x: [f'background-color: {mol_color[v]}' for v in x],
                               axis=1)
# before printing surround the values with \ce{} to make them chemical formulas
# adjust column names to be Cluster 1, Cluster 2, etc.
# write the dataframe to a latex table
mol_df.to_latex(paper_path.joinpath('cluster_molecule_table.tex'),
                multicol_align='|c|',
                hrules=True,
                convert_css=True,
                )

mol_df



In [62]:
august_database.iso_data